In [2]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum

yu.setpath('general')

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
enss=['b','c','d','e']

In [4]:
# Q2 vs msq when sink=0
fig, axs = yu.getFigAxs(1,1,Lrow=4,Lcol=12)
ax=axs[0,0]

max_mom2_pc=81
range_xyz=range(-int(np.sqrt(max_mom2_pc))-1,int(np.sqrt(max_mom2_pc))+2)
moms_pc=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2_pc]

moms=[pf+pc for pf in [[0,0,0]] for pc in moms_pc]
moms=[list(ele) for ele in set([tuple(yum.mom2standard(mom)) for mom in moms])]; moms.sort()
msqs=[mom[3]**2+mom[4]**2+mom[5]**2 for mom in moms]

msq_1=[np.max([msqs[i] for i in range(len(moms)) if yum.mom2Q2(moms[i],ens)<=1]) for ens in enss]
msq_1p5=[np.max([msqs[i] for i in range(len(moms)) if yum.mom2Q2(moms[i],ens)<=1.5]) for ens in enss]
msq_2=[np.max([msqs[i] for i in range(len(moms)) if yum.mom2Q2(moms[i],ens)<=2]) for ens in enss]

for iens,ens in enumerate(enss):
    Q2s=[yum.mom2Q2(mom,ens) for mom in moms]
    plt_x=msqs; plt_y=Q2s
    ax.plot(plt_x,plt_y,color=yu.colors8[iens],label=f'{ens}: L={yu.ens2NL[ens]*yu.ens2a[ens]:.3} fm; [{msq_1[iens]},{msq_1p5[iens]},{msq_2[iens]}]',ms=4)

yu.addRefLine(ax,1)
yu.addRefLine(ax,2)
ax.legend()
ax.set_xlabel('msq')
ax.set_ylabel(r'$Q^2$ [GeV$^2$]')
fig.suptitle('Integers in legend indicate msq cuts for Q2 cuts at 1,1.5,2')
yu.finalizePlot('Q2_msq')

In [ ]:
ens='b'
def getRank(moms,case):
    if case=='unequal':
        mpirs=[(mom,proj,insert,ri) for mom in moms for proj in projs for insert in inserts for ri in [0,1] if insert[0]!=insert[1] and yum.useQ(mom,proj,insert)[ri]]
        # print(mpirs)
    elif case=='equal':
        mpirs=[(mom,proj,insert,ri) for mom in moms for proj in projs for insert in inserts for ri in [0,1] if insert[0]==insert[1] and yum.useQ(mom,proj,insert)[ri]]
    elif case=='all':
        mpirs=[(mom,proj,insert,ri) for mom in moms for proj in projs for insert in inserts for ri in [0,1] if yum.useQ(mom,proj,insert)[ri]]
    G=np.array([yum.funcs_ri[ri](yum.ME2FF(939/yu.ens2aInv[ens],np.array(np.array(mom[:3])+mom[3:])*(2*np.pi/yu.ens2NL[ens]),(np.array(mom[:3]))*(2*np.pi/yu.ens2NL[ens]),proj,insert)) for mom,proj,insert,ri in mpirs])
    if len(G)!=0:
        U, S, VT = np.linalg.svd(G)
        tol = 1e-10
        rank = np.sum(S > tol)
        return rank
    return 0
def getRanks(moms):
    return [getRank(moms,case) for case in ['unequal','equal','all']]

res=yu.load_pkl_reg('n2qpp1s,2moms,2ranks')
if res is not None:
    n2qpp1s,n2qpp1_to_moms,n2qpp1_to_ranks=res
else:
    max_mom2_pc=27
    range_xyz=range(0,int(np.sqrt(max_mom2_pc))+2)
    moms_pc=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2_pc]

    max_mom2_pf=10
    range_xyz=range(-int(np.sqrt(max_mom2_pf))-1,int(np.sqrt(max_mom2_pf))+2)
    moms_pf=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2_pf]

    moms=[pf+pc for pf in moms_pf for pc in moms_pc]
    moms=[list(ele) for ele in set([tuple(yum.mom2standard_exchangeSourceSink(mom)) for mom in moms])]; moms.sort()
    n2qpp1s_moms=[yum.mom2n2qpp1_sym(mom) for mom in moms]
    n2qpp1s=sorted(yu.removeDuplicates(n2qpp1s_moms), key=lambda x: yum.n2qpp12Q2(x,ens))

    n2qpp1_to_moms={n2qpp1:[] for n2qpp1 in n2qpp1s}
    for mom in moms:
        n2qpp1_to_moms[yum.mom2n2qpp1_sym(mom)].append(mom)

    n2qpp1_to_ranks={}
    for i,n2qpp1 in enumerate(n2qpp1s):
        print(f'{i}/{len(n2qpp1s)}',end='          \r')
        moms=n2qpp1_to_moms[n2qpp1]
        n2qpp1_to_ranks[n2qpp1]=getRanks(moms)
        
    yu.save_pkl_reg('n2qpp1s,2moms,2ranks',[n2qpp1s,n2qpp1_to_moms,n2qpp1_to_ranks])
    
    txts=[f'(n2q,n2p,n2p1) [Q2*1000 for {",".join(enss)}]: rank for [unequal,equal,all]; moms']
    for n2qpp1 in n2qpp1s:
        Q2s=','.join([f'{int(yum.n2qpp12Q2(n2qpp1,ens)*1000): 4d}' for ens in enss])
        txt=f'{n2qpp1} [{Q2s}]: {n2qpp1_to_ranks[n2qpp1]}; {n2qpp1_to_moms[n2qpp1]}'
        txts.append(txt)
    yu.save_txt_reg('n2qpp1s,2moms,2ranks',txts)

ranks_all=yu.removeDuplicates([tuple(n2qpp1_to_ranks[n2qpp1]) for n2qpp1 in n2qpp1s])
print(ranks_all)

[(0, 1, 1), (1, 1, 1), (1, 2, 3), (2, 1, 3), (2, 2, 3), (2, 3, 3), (3, 2, 3), (3, 3, 3)]


In [53]:
def ranks2fillstyle(ranks):
    r1,r2,r3=ranks
    t=(r1==3,r2==3,r3==3)
    
    fillstyle={
        (True,True,True):'full', # rank3 for both unequal and equal (therefore also all)
        (True,False,True):'left', # rank3 for unequal
        (False,True,True):'right', # rank3 for equal
        (False,False,True):'bottom', # rank3 for all
        (False,False,False):'none', # rank3 for no cases
    }[t]
    return fillstyle
fillstyles=['full','left','right','bottom','none']

fig, axs = yu.getFigAxs(1,1,Lrow=20,Lcol=30)
ax=axs[0,0]
ax.set_xlabel(r'$Q^2$ [GeV$^2$]')
ax.set_ylabel(r'Ensemble')
ax.set_ylim([-0.1,3.6])
ax.set_yticks([0,1,2,3])
ax.set_yticklabels(enss)

ax.set_xlim([-0.02,1.02])

p12s=range(11)

for iens,ens in enumerate(enss):
    for p12 in p12s:
        n2qpp1s_p12=[n2qpp1 for n2qpp1 in n2qpp1s if n2qpp1[-1]==p12]
        for fillstyle in fillstyles:
            n2qpp1s_p12_fillstyle=[n2qpp1 for n2qpp1 in n2qpp1s_p12 if ranks2fillstyle(n2qpp1_to_ranks[n2qpp1])==fillstyle]
            Q2s=[yum.n2qpp12Q2(n2qpp1,ens) for n2qpp1 in n2qpp1s_p12_fillstyle]
            
            xs=Q2s; ys=[iens+p12*0.05]*len(Q2s)
            ax.plot(xs,ys,color='b', marker=mpl.markers.MarkerStyle("s", fillstyle=fillstyle))
yu.finalizePlot('momDistribution')

In [3]:
# def useMom(mom):
#     px,py,pz=mom[3:]
#     if pz>=py>=pz>=0:
#         return True
#     return False
    
# fig, axs = yu.getFigAxs(1,1,Lrow=6,Lcol=18)
# ax=axs[0,0]
# ax.set_xlabel(r'$Q^2$ [GeV$^2$]')
# # ax.set_xlim([-0.1,1.1])
# # ax.set_xlim([-0.05,0.6])
# ax.set_ylabel(r'Ensemble')
# ax.set_ylim([-1,4])
# ax.set_yticks([0,1,2,3])
# ax.set_yticklabels(enss)

# for iens,ens in enumerate(enss):
#     max_mom2_pc={'b':16,'c':16,'d':16,'e':16}[ens]
#     range_xyz=range(-int(np.sqrt(max_mom2_pc))-1,int(np.sqrt(max_mom2_pc))+2)
#     moms_pc=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2_pc]

#     max_mom2_pf={'b':5,'c':5,'d':5,'e':5}[ens]
#     range_xyz=range(-int(np.sqrt(max_mom2_pf))-1,int(np.sqrt(max_mom2_pf))+2)
#     moms_pf=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2_pf]
    
#     colors=['r','g','b','orange','cyan','purple']
#     for ipf2max in range(max_mom2_pf+1):
#         moms=[pf+pc for pf in moms_pf for pc in moms_pc if pf[0]**2+pf[1]**2+pf[2]**2==ipf2max]
        
#         Q2s_all=[round(yu.mom2Q2(mom,ens),5) for mom in moms]
#         Q2s=list(set(Q2s_all))
#         Q2s.sort()
        
#         Q2s_rank3=[]
#         for iQ2,Q2 in enumerate(Q2s):
#             # print(f'{ens},{ipf2max},{iQ2}/{len(Q2s)}',end='          \r')
#             inds=[i for i,ele in enumerate(Q2s_all) if ele==Q2]
#             t_moms=[moms[ind] for ind in inds if useMom(moms[ind])]
            
#             mpirs=[(mom,proj,insert,ri) for mom in t_moms for proj in projs for insert in inserts for ri in [0,1] if insert[0]!=insert[1] and yum.useQ_noAvg(mom,proj,insert)[ri]]
#             G=np.array([yum.funcs_ri[ri](yum.ME2FF(938/yu.ens2aInv[ens],np.array(np.array(mom[:3])+mom[3:])*(2*np.pi/yu.ens2NL[ens]),(np.array(mom[:3]))*(2*np.pi/yu.ens2NL[ens]),proj,insert)) for mom,proj,insert,ri in mpirs])
#             if len(G)!=0:
#                 U, S, VT = np.linalg.svd(G)
#                 tol = 1e-10
#                 rank = np.sum(S > tol)
#                 if rank==3:
#                     Q2s_rank3.append(Q2)
        
#         ys=[iens+(ipf2max-max_mom2_pf//2)*0.08]*len(Q2s)
#         ax.scatter(Q2s,ys,color=colors[ipf2max] if ipf2max<len(colors) else None, label=ipf2max if iens==0 else None,facecolor='white')
        
#         ys=[iens+(ipf2max-max_mom2_pf//2)*0.08]*len(Q2s_rank3)
#         ax.scatter(Q2s_rank3,ys,color=colors[ipf2max] if ipf2max<len(colors) else None, marker=mpl.markers.MarkerStyle("o", fillstyle="left"))
        
#         break

# ax.legend(fontsize=16,title='sink mom');